# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

- **Consider a significance level of 5% for all tests.**

In [1]:
# import numpy and pandas
import numpy as np
import pandas as pd
import scipy.stats as st

# Challenge 1 - Independent Sample T-tests

In this challenge, we will be using the Pokemon dataset. Before applying statistical methods to this data, let's first examine the data.

To load the data, run the code below.

In [2]:
# Run this code:
pokemon = pd.read_csv('../pokemon.csv')

Let's start off by looking at the `head` function in the cell below.

In [3]:
# Your code here:
pokemon.head()

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False


The first thing we would like to do is compare the legendary Pokemon to the regular Pokemon. To do this, we should examine the data further. What is the count of legendary vs. non legendary Pokemons?

In [4]:
# Your code here:
cnt_leg = len(pokemon[pokemon['Legendary']==True])
cnt_non_leg = len(pokemon[pokemon['Legendary']==False])
print('Legendary count:',cnt_leg)
print('Non-Legendary count:',cnt_non_leg)

Legendary count: 65
Non-Legendary count: 735


Compute the mean and standard deviation of the total points for both legendary and non-legendary Pokemon.

In [5]:
# Your code here:
mean_leg = pokemon['Total'][pokemon['Legendary']==True].mean()
mean_non_leg = pokemon['Total'][pokemon['Legendary']==False].mean()
std_leg = pokemon['Total'][pokemon['Legendary']==True].std(ddof=0)
std_non_leg = pokemon['Total'][pokemon['Legendary']==False].std(ddof=0)
print('Mean Legendary:',mean_leg)
print('Mean Non-Legendary:',mean_non_leg)
print('Std Legendary:',std_leg)
print('Std Non-Legendary:',std_non_leg)

Mean Legendary: 637.3846153846154
Mean Non-Legendary: 417.21360544217686
Std Legendary: 60.46682302213779
Std Non-Legendary: 106.68776660321979


The computation of the mean might give us a clue regarding how the statistical test may turn out; However, it certainly does not prove whether there is a significant difference between the two groups.

In the cell below, use the `ttest_ind` function in `scipy.stats` to compare the the total points for legendary and non-legendary Pokemon. Since we do not have any information about the population, assume the variances are not equal.

In [6]:
# Your code here:
st.ttest_ind(pokemon['Total'][pokemon['Legendary']==True], pokemon['Total'][pokemon['Legendary']==False], equal_var=False)

Ttest_indResult(statistic=25.8335743895517, pvalue=9.357954335957446e-47)

What do you conclude from this test? Write your conclusions below.

In [7]:
# Your conclusions here:
# Underlying H0: means of Total point of legendary vs. non-legendary are the same
# H0 rejected due to low p value -> means are significantly different.

How about we try to compare the different types of pokemon? In the cell below, list the types of Pokemon from column `Type 1` and the count of each type.

In [8]:
# Your code here:
pokemon_type1_cnt_mean_std = pokemon.pivot_table(values=['Total'], index=['Type 1'], aggfunc=['count','mean','std']).reset_index()
pokemon_type1_cnt_mean_std

,Type 1,count,mean,std
,,Total,Total,Total
0,Bug,69,378.927536,117.875223
1,Dark,31,445.741935,109.126217
2,Dragon,32,550.531250,146.267538
3,Electric,44,443.409091,105.721952
4,Fairy,17,413.176471,123.781680
5,Fighting,27,416.444444,102.464378
6,Fire,52,458.076923,109.760496
7,Flying,4,485.000000,161.400124
8,Ghost,32,439.562500,110.072685


Since water is the largest group of Pokemon, compare the mean and standard deviation of water Pokemon to all other Pokemon.

In [9]:
# Your code here:

# compare?

pokemon_type1_cnt_mean_std['Water mean'] = pokemon['Total'][pokemon['Type 1'] == 'Water'].mean()
pokemon_type1_cnt_mean_std['Water std'] = pokemon['Total'][pokemon['Type 1'] == 'Water'].std()
pokemon_type1_cnt_mean_std['Delta mean'] = pokemon_type1_cnt_mean_std[('mean','Total')] - pokemon_type1_cnt_mean_std['Water mean']
pokemon_type1_cnt_mean_std['Delta std'] = pokemon_type1_cnt_mean_std[('std','Total')] - pokemon_type1_cnt_mean_std['Water std']

pokemon_type1_cnt_mean_std

,Type 1,count,mean,std,Water mean,Water std,Delta mean,Delta std
,,Total,Total,Total,,,,
0,Bug,69,378.927536,117.875223,430.455357,113.188266,-51.527821,4.686957
1,Dark,31,445.741935,109.126217,430.455357,113.188266,15.286578,-4.062049
2,Dragon,32,550.531250,146.267538,430.455357,113.188266,120.075893,33.079271
3,Electric,44,443.409091,105.721952,430.455357,113.188266,12.953734,-7.466314
4,Fairy,17,413.176471,123.781680,430.455357,113.188266,-17.278887,10.593414
5,Fighting,27,416.444444,102.464378,430.455357,113.188266,-14.010913,-10.723888
6,Fire,52,458.076923,109.760496,430.455357,113.188266,27.621566,-3.427770
7,Flying,4,485.000000,161.400124,430.455357,113.188266,54.544643,48.211858
8,Ghost,32,439.562500,110.072685,430.455357,113.188266,9.107143,-3.115581


Perform a hypothesis test comparing the mean of total points for water Pokemon to all non-water Pokemon. Assume the variances are equal. 

In [10]:
# Your code here:
results = pd.DataFrame(columns=['Type 1','stat','p-value'])
for type1 in pokemon_type1_cnt_mean_std['Type 1']:
    stat, p = st.ttest_ind(pokemon['Total'][pokemon['Type 1']==type1], pokemon['Total'][pokemon['Type 1']=='Water'], equal_var=False)
    results = results.append({'Type 1':type1, 'stat':stat, 'p-value':p}, ignore_index=True)
results
pokemon_type1_cnt_mean_std = pd.merge(pokemon_type1_cnt_mean_std, results, left_index=True, right_index=True, how='inner')
pokemon_type1_cnt_mean_std['H0'] = np.where(pokemon_type1_cnt_mean_std['p-value']<=0.05, 'Rejected', 'Not Rejected')
pokemon_type1_cnt_mean_std

/Users/uli/anaconda3/envs/ironhack/lib/python3.8/site-packages/pandas/core/reshape/merge.py:643: UserWarning: merging between different levels can give an unintended result (2 levels on the left,1 on the right)
  warnings.warn(msg, UserWarning)


,"(Type 1, )","(count, Total)","(mean, Total)","(std, Total)","(Water mean, )","(Water std, )","(Delta mean, )","(Delta std, )",Type 1,stat,p-value,H0
0,Bug,69,378.927536,117.875223,430.455357,113.188266,-51.527821,4.686957,Bug,-2.899769,0.004339,Rejected
1,Dark,31,445.741935,109.126217,430.455357,113.188266,15.286578,-4.062049,Dark,0.684640,0.496774,Not Rejected
2,Dragon,32,550.531250,146.267538,430.455357,113.188266,120.075893,33.079271,Dragon,4.291278,0.000102,Rejected
3,Electric,44,443.409091,105.721952,430.455357,113.188266,12.953734,-7.466314,Electric,0.674880,0.501609,Not Rejected
4,Fairy,17,413.176471,123.781680,430.455357,113.188266,-17.278887,10.593414,Fairy,-0.542173,0.593610,Not Rejected
5,Fighting,27,416.444444,102.464378,430.455357,113.188266,-14.010913,-10.723888,Fighting,-0.624567,0.535580,Not Rejected
6,Fire,52,458.076923,109.760496,430.455357,113.188266,27.621566,-3.427770,Fire,1.484796,0.140672,Not Rejected
7,Flying,4,485.000000,161.400124,430.455357,113.188266,54.544643,48.211858,Flying,0.670035,0.549286,Not Rejected
8,Ghost,32,439.562500,110.072685,430.455357,113.188266,9.107143,-3.115581,Ghost,0.410159,0.683399,Not Rejected
9,Grass,70,421.142857,106.650626,430.455357,113.188266,-9.312500,-6.537640,Grass,-0.559655,0.576532,Not Rejected


Write your conclusion below.

In [11]:
# Your conclusions here:
# Underlying H0: means of Total point of type1 non-water vs. type1 water are the same
# Conclusion: H0 rejected due to low p value for Bug, Dragon, Psychic & Steel -> means are significantly different.
# Remaing are not different enough!

# Challenge 2 - Matched Pairs Test

In this challenge we will compare dependent samples of data describing our Pokemon. Our goal is to see whether there is a significant difference between each Pokemon's defense and attack scores. Our hypothesis is that the defense and attack scores are equal. In the cell below, import the `ttest_rel` function from `scipy.stats` and compare the two columns to see if there is a statistically significant difference between them.

In [12]:
# Your code here:
st.ttest_rel(pokemon['Attack'],pokemon['Defense'])

Ttest_relResult(statistic=4.325566393330478, pvalue=1.7140303479358558e-05)

Describe the results of the test in the cell below.

In [13]:
# Your conclusions here:
# Underlying H0: Attack and Defense score the same
# p-value very low -> reject H0 so it can be concluded they differ significantly

We are also curious about whether therer is a significant difference between the mean of special defense and the mean of special attack. Perform the hypothesis test in the cell below. 

In [14]:
# Your code here:
st.ttest_rel(pokemon['Sp. Atk'],pokemon['Sp. Def'])

Ttest_relResult(statistic=0.853986188453353, pvalue=0.3933685997548122)

Describe the results of the test in the cell below.

In [15]:
# Your conclusions here:
# Underlying H0: Attack and Defense score the same
# p-value pretty high -> H0 can not be rejected, so it can be concluded they do not differ significantly

As you may recall, a two sample matched pairs test can also be expressed as a one sample test of the difference between the two dependent columns.

Import the `ttest_1samp` function and perform a one sample t-test of the difference between defense and attack. Test the hypothesis that the difference between the means is zero. Confirm that the results of the test are the same.

In [16]:
# Your code here:
# Generate delta column
pokemon['Dif Att Def'] = pokemon['Attack']-pokemon['Defense']    
# perform sample test
st.ttest_1samp(pokemon['Dif Att Def'],0)
# Same same than above

Ttest_1sampResult(statistic=4.325566393330478, pvalue=1.7140303479358558e-05)

# Bonus Challenge - The Chi-Square Test

The Chi-Square test is used to determine whether there is a statistically significant difference in frequencies. In other words, we are testing whether there is a relationship between categorical variables or rather when the variables are independent. This test is an alternative to Fisher's exact test and is used in scenarios where sample sizes are larger. However, with a large enough sample size, both tests produce similar results. Read more about the Chi Squared test [here](https://en.wikipedia.org/wiki/Chi-squared_test).

In the cell below, create a contingency table using `pd.crosstab` comparing whether a Pokemon is legenadary or not and whether the Type 1 of a Pokemon is water or not.

In [17]:
# Your code here:
pokemon['Type 1 = Water?'] = np.where(pokemon['Type 1'] == 'Water', 1, 0)
cross_leg_wat = pd.crosstab(pokemon['Legendary'], pokemon['Type 1 = Water?'])
cross_leg_wat

Type 1 = Water?,0,1
Legendary,,
False,627,108
True,61,4


Perform a chi-squared test using the `chi2_contingency` function in `scipy.stats`. You can read the documentation of the function [here](https://docs.scipy.org/doc/scipy-0.15.1/reference/generated/scipy.stats.chi2_contingency.html).

In [18]:
# Your code here:
st.chi2_contingency(cross_leg_wat)
#return: statistic, p-value, degrees of freedom, expected probs if H0 holds

(2.9429200762850503,
 0.08625467249550949,
 1,
 array([[632.1, 102.9],
        [ 55.9,   9.1]]))

Based on a 95% confidence, should we reject the null hypothesis?

In [19]:
# Your answer here:
# p-value is 0.086, so based on a 95% confidence we cannot reject H0
# Underlying H0: Having type Water does not influence if being legendary or not
# conclusion: at legendary pokemon there are more of type water (?)